## Parking Controller via MPC with Obstracles

###System dynamics:
\begin{align}
\dot{x}	&=&	Vcos(\phi+\beta)\\
\dot{y}	&=&	Vsin(\phi+\beta)\\
\dot{\phi}	&=&	V/l sin(\beta)\\
\dot{V}	&=&	a
\end{align}
Here $a$ is the center of mass acceleration
$z = [x,y,\phi,V]$, $u = [\beta, a]$


In [1]:
#Pkg.add("JuMP"); Pkg.add("Ipopt"); Pkg.add("Gadfly"); Pkg.add("Interact")
#Pkg.checkout("JuMP") # need latest master for matrices
using JuMP, Ipopt
using PyPlot
#using ODE
using PyCall

function solveMPC(l,w,n,m,T,z0,zT,zmax,umax,dt,Nb)
    mpc = Model(solver=IpoptSolver(print_level=0))
    @defVar(mpc, -zmax[i] <= z[i=1:n,t=0:T] <= zmax[i])
    @defVar(mpc, -umax[i] <= u[i=1:m,t=0:T] <= umax[i])
    # Cost
    #@setObjective(mpc, Min,
    #    sum{100*(z[1,t]+z[2,t]+z[3,t]+z[4,t])^2 + sum{u[j,t]^2,j=1:m},t=0:T})
    @setObjective(mpc, Min,
    sum{(z[1,T]-zT[1])^2+(z[2,T]-zT[2])^2+(z[3,T]-zT[3])^2,t=T-1:T})

    # Link state and control across the horizon
    for t = 0:T-1        
        @addNLConstraint(mpc, z[1,t+1] == z[1,t] + dt*z[4,t]*cos(z[3,t]+u[1,t]))
        @addNLConstraint(mpc, z[2,t+1] == z[2,t] + dt*z[4,t]*sin(z[3,t]+u[1,t]))
        @addNLConstraint(mpc, z[3,t+1] == z[3,t] + dt*z[4,t]/l*sin(u[1,t]))
        @addNLConstraint(mpc, z[4,t+1] == z[4,t] + dt*u[2,t])
        # Blocking Move (quick/dirty version)
        if (t>0) && (rem(t,Nb)!=0)
            @addConstraint(mpc, u[:,t] .== u[:,t-1])
        end
    end

    # Obstacle avoidance constraints 
    if 0<1
        @defVar(mpc, lambda[i=1:16,t=0:T] >= 0)
        for t = 0:Nb:T-1 
            # Four corners of the car
            #A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
            #B=[x-l*cos(psi)+w*sin(psi),y-l*sin(psi)-w*cos(psi)]
            #C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
            #D=[x-l*cos(psi)-w*sin(psi),y-l*sin(psi)+w*cos(psi)]

            #To be parametrized later : parallel parking bay with two obstacles
            # x>=xl OR y-yt>=0 
            # x<=xr OR y-yt>=0 
            # Evaluated at A,B,C,D
            xl=-3
            xr=3
            yt=0

            @addNLConstraint(mpc, lambda[1,t]*(z[1,t]+l*cos(z[3,t])+w*sin(z[3,t])-xl)+lambda[2,t+1]*(z[2,t]+l*sin(z[3,t])-w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[1,t]+lambda[2,t] == 1)
            @addNLConstraint(mpc, lambda[3,t]*(z[1,t]-l*cos(z[3,t])+w*sin(z[3,t])-xl)+lambda[4,t+1]*(z[2,t]-l*sin(z[3,t])-w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[3,t]+lambda[4,t] == 1)
            @addNLConstraint(mpc, lambda[5,t]*(z[1,t]+l*cos(z[3,t])-w*sin(z[3,t])-xl)+lambda[6,t+1]*(z[2,t]+l*sin(z[3,t])+w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[5,t]+lambda[6,t] == 1)
            @addNLConstraint(mpc, lambda[7,t]*(z[1,t]-l*cos(z[3,t])-w*sin(z[3,t])-xl)+lambda[8,t+1]*(z[2,t]-l*sin(z[3,t])+w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[7,t]+lambda[8,1] == 1)

            @addNLConstraint(mpc, lambda[9,t]*(-(z[1,t]+l*cos(z[3,t])+w*sin(z[3,t]))+xr)+lambda[10,t+1]*(z[2,t]+l*sin(z[3,t])-w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[9,t]+lambda[10,t] == 1)
            @addNLConstraint(mpc, lambda[11,t]*(-(z[1,t]-l*cos(z[3,t])+w*sin(z[3,t]))+xr)+lambda[12,t+1]*(z[2,t]-l*sin(z[3,t])-w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[11,t]+lambda[12,t] == 1)
            @addNLConstraint(mpc, lambda[13,t]*(-(z[1,t]+l*cos(z[3,t])-w*sin(z[3,t]))+xr)+lambda[14,t+1]*(z[2,t]+l*sin(z[3,t])+w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[13,t]+lambda[14,t] == 1)
            @addNLConstraint(mpc, lambda[15,t]*(-(z[1,t]-l*cos(z[3,t])-w*sin(z[3,t]))+xr)+lambda[16,t+1]*(z[2,t]-l*sin(z[3,t])+w*cos(z[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[15,t]+lambda[16,1] == 1)

        end
    end

    # Initial conditions
    @addConstraint(mpc, z[:,0] .== z0)
    # Final state
    #@addConstraint(mpc, z[:,T] .== zT)
    # Solve the NLP
    solve(mpc)
    # Return the control plan
    # return getValue(u[:,0])
    #return getValue(u[:,0]), getValue(z[:,1])
    return getValue(u), getValue(z)
end

INFO: Loading help data...


solveMPC (generic function with 1 method)

In [2]:
function zdot_fun(z,u,l)
    beta=u[1]
    a=u[2]
    phi=z[3]
    V=z[4]
    xdot=V*cos(phi+beta)
    ydot=V*sin(phi+beta)
    phidot=V/l*sin(beta)
    Vdot=a                      
    zdot = [xdot;ydot;phidot;Vdot]
    return zdot
end


function simulate_car_MPC(T,Tsim,Tr,z0,zT,zmax,umax,l,n,m,dt)
    u_history = zeros(m,Tsim)
    z_history = zeros(n,Tsim)
    z_t = z0[:]
    for t = 0:Tr:Tsim-1
        # SHRINKING HORIZON MPC
        u_vec, z_vec= solveMPC(l,n,m,T-t,z_t,zT,zmax,umax,dt)
        print("-MPC solved-")
        for k = 1:min(Tr,length(u_vec))
              u_t=u_vec[:,k]
              z_history[:,t+k] = z_t[:]
              u_history[:,t+k] = u_t[:]
              z_t = z_t + dt*zdot_fun(z_t,u_t,l)
        end
    end
    return  u_history, z_history
end



simulate_car_MPC (generic function with 1 method)

### Run Simulations

In [ ]:
#Car Lengths to be fixed later
#l = 1.91 
#lf=1.91
l = 1.738
lr=3
width=0.93
#width_bay = 6.00
#height_bay = 2.5

T = 600  # MPC horizon
dt=0.1
Tsim = T  # Length of time we simulate
Treplanning = 100  

#Case 1
z0 = [6;6;pi;0]  
zT = [0;-1.1;0;0]  

#Case 2
#z0 = [6;6;pi;0]  
#zT = [0;-1.1;pi;0] 

zmax=[100;100;pi;5*dt]
umax=[0.6;1*dt^2]
zmax=[100;100;pi;10*dt]
umax=[0.6;2*dt^2]
n=4
m=2

# Closed- Loop MPC, NOT TESTED FOR NOW
#u_vec,z_vec = simulate_car_MPC(T,Tsim,Treplanning,z0,zT,zmax,umax,l,n,m,dt)

# Open Loop MPC
Nb=10 # blocking move
u_vec, z_vec= solveMPC(l,width,n,m,T,z0,zT,zmax,umax,dt,Nb)





###Next cell plots the parking manouver

In [6]:
@pyimport matplotlib.pyplot as plt

#Find Axis Limits
xmin = minimum([z_vec[1,:]])
xmax = maximum([z_vec[1,:]])
xmin, xmax = xmin - 0.1(xmax-xmin), xmax+ 0.1*(xmax-xmin)
ymin = minimum([z_vec[2,:]])
ymax = maximum([z_vec[2,:]])
ymin, ymax = ymin - 0.1(ymax-ymin), ymax+ 0.1*(ymax-ymin)

function plot_car(x,y,psi,beta,l,w,pt)
    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-l*cos(psi)+w*sin(psi),y-l*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-l*cos(psi)-w*sin(psi),y-l*sin(psi)+w*cos(psi)]
    #wheels
    lw=0.4
    E=[A[1]+lw*cos(psi+beta),A[2]+lw*sin(psi+beta)]
    F=[C[1]+lw*cos(psi+beta),C[2]+lw*sin(psi+beta)]
    
    pt.plot([B[1] ,A[1]],[B[2], A[2]],"bo-")
    pt.plot([C[1] ,D[1]],[C[2], D[2]],"bo-")
    pt.plot([C[1] ,A[1]],[C[2], A[2]],"bo-")
    pt.plot([B[1] ,D[1]],[B[2], D[2]],"bo-")
    pt.plot([A[1] ,E[1]],[A[2], E[2]],"go-")
    pt.plot([C[1] ,F[1]],[C[2], F[2]],"go-")
end


#Construct Figure and Plot Data
fig = figure()
ax = plt.axes()
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
plt.plot(transpose(z_vec[1,:]),transpose(z_vec[2,:]), "r-")
for i=1:10:T
    plot_car(z_vec[1,i],z_vec[2,i],z_vec[3,i],u_vec[1,i],l,width,plt)
end
plt.plot([-3 ,-3],[-3, 0],"r-")
plt.plot([3 ,3],[-3, 0],"r-")
plt.plot([-10 ,-3],[0, 0],"r-")
plt.plot([3 ,15],[0, 0],"r-")
plt.show()




#Plot start and end points
#ax[:plot](z_vec[1,1],z_vec[2,1], "ro")
#ax[:plot](z_vec[1,end],z_vec[2,end], "rs")

### Next makes a video of the manouver

In [5]:
#Pkg.add("PyPlot")
#Pkg.add("PyCall")
#Pkg.add("VideoIO")
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim

pygui(true)

# First set up the figure, the axis, and the plot element we want to animate
fig = figure()
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
#ax = plt.axes(xlim=(-10, 5), ylim=(-2, 10))
ax = plt.axes(xlim=(-15, 15), ylim=(-15, 15))
#ax2 = plt.axes()
global line1 = ax[:plot]([], [], "ro-")[1]
global line2 = ax[:plot]([], [], "ro-")[1]
global line3 = ax[:plot]([], [], "ro-")[1]
global line4 = ax[:plot]([], [], "ro-")[1]
global line5 = ax[:plot]([], [], "go-")[1]
global line6 = ax[:plot]([], [], "go-")[1]

# To plot parking bay.. do not have time
#global line7 = ax[:plot]([-3 ,-3],[-3, 0],"r-")
#plt.plot([-3 ,-3],[-3, 0],"r-")
#plt.plot([3 ,3],[-3, 0],"r-")
#plt.plot([-10 ,-3],[0, 0],"r-")
#plt.plot([3 ,15],[0, 0],"r-")

function points_car(x,y,psi,beta,l,w)
    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-l*cos(psi)+w*sin(psi),y-l*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-l*cos(psi)-w*sin(psi),y-l*sin(psi)+w*cos(psi)]
    #wheels
    lw=0.4
    E=[A[1]+lw*cos(psi+beta),A[2]+lw*sin(psi+beta)]
    F=[C[1]+lw*cos(psi+beta),C[2]+lw*sin(psi+beta)]
    return A,B,C,D,E,F
end
 

# initialization function: plot the background of each frame
function init()
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    line1[:set_data]([], [])
    line2[:set_data]([], [])
    line3[:set_data]([], [])
    line4[:set_data]([], [])
    line5[:set_data]([], [])
    line6[:set_data]([], [])
    return (line1,line2,line3,line4,line5,line6,None)
end

# animation function.  This is called sequentially
function animate(i)
    k=i+1
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    A,B,C,D,E,F=points_car(z_vec[1,k],z_vec[2,k],z_vec[3,k],u_vec[1,k],l,width)
    line1[:set_data]([B[1] ,A[1]],[B[2], A[2]])
    line2[:set_data]([C[1] ,D[1]],[C[2], D[2]])
    line3[:set_data]([C[1] ,A[1]],[C[2], A[2]])
    line4[:set_data]([B[1] ,D[1]],[B[2], D[2]])
    line5[:set_data]([A[1] ,E[1]],[A[2], E[2]])
    line6[:set_data]([C[1] ,F[1]],[C[2], F[2]])
    return (line1,line2,line3,line4,line5,line6,None)
end

# call the animator.  blit=True means only re-draw the parts that have changed.
myanim = anim.FuncAnimation(fig, animate, init_func=init, frames=T-1, interval=100)
#myanim[:save]("./parking.mp4")

#myanim[:save]("./parking.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
#myanim[:save]("plots-julia/PyPlots-sinplot.mp4", extra_args=["-vcodec"])

#myanim[:save]("./sinplot.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])

# call our new function to display the animation
#display("text/html", string("""<video autoplay controls><source src="data:video/x-m4v;base64,""",
#                            base64(open(readbytes,"./sinplot.mp4")),"""" type="video/mp4"></video>"""))

PyObject <matplotlib.animation.FuncAnimation object at 0x0000000020B0A128>